# Merton Jump Diffusion Process: Option Pricing with Monte Carlo

This notebook describes how to use a Merton jump diffusion process to price European options using Monte Carlo simulation. 

With the inclusion of a normal compound Poisson process in the geometric Brownian motion and a term added to the risk-neutral drift, the implementation is similar to that of the Black-Scholes model.

### STEP 1. Initialization of the parameter:

Establish the parameters, including the jump component parameters (mu_j, sigma_j, lmbda), maturity period (T), spot price (S0), strike price (K), risk-free interest rate (r), dividend rate (q), volatility (sigma), and volatility (sigma).

### STEP 2. Compute Risk-Neutral Measure:

To guarantee fair option pricing, compute the risk-neutral measure (muRN), which accounts for risk. This entails figuring out the asset's risk-neutral drift.

### STEP 3. Monte Carlo Parameters and Initialization:

Indicate the relevant Monte Carlo simulation parameters, such as the number of blocks (nblocks) and samples (nsample) per block.

Set up arrays with initial values to hold the put and call option prices that have been calculated.

### STEP 4. Implementation of Monte Carlo Simulation:

Run the loop of the Monte Carlo simulation:
For every block in the simulation:
Provide random variables for the Merton jump diffusion process's jump and diffusion components.
Put the diffusion and leap components together to get the stock price trajectory.
Determine each sample's option payoff at maturity.
By averaging the discounted payoffs, find the option price for the block.
The option prices that were obtained for each block should be kept in the appropriate arrays.

### STEP 5. Calculation of the Final Option Price:

By summing the prices acquired from each block, determine the final call and put option prices.

In [4]:
import numpy as np

# Parameters
T = 128.0/365.0      # Maturity (in years)
S0 = 21730     # Spot price (current price)
K = 22200     # Strike price (agreed price for buying or selling)
r = 0.05   # Risk-free interest rate
q = 0.0   # Dividend rate (rate of return paid on the stock)

# Model parameters
sigma = 0.02 # Volatility (degree of variation in the stock's price)

# Jump component parameters (Normal Compound Poisson Process - NCPP)
mu_j = -0.01     # Mean of the jump component
sigma_j = 0.02  # Volatility of the jump component
lmbda = 0.005     # Jump frequency (intensity of jumps)

# Calculate the risk-neutral measure (drift)
muRN = r - q - 0.5 * sigma ** 2 - lmbda * (np.exp(mu_j + 0.5 * sigma_j ** 2) - 1)

# Monte Carlo parameters
nblocks = 30000  # Number of simulation blocks
nsample = 10000   # Number of samples per block

# Initialize arrays to store option prices
Vc_list = np.zeros(nblocks)  # Call option prices
Vp_list = np.zeros(nblocks)  # Put option prices

# Monte Carlo simulation
for i in range(nblocks):
    # Generate random variables for the diffusion component
    A = muRN * T + sigma * np.sqrt(T) * np.random.normal(size=(1, nsample))
    
    # Generate random variables for the jump component
    N = np.random.poisson(lmbda, size=(1, nsample))
    J = mu_j * N + sigma_j * np.sqrt(N) * np.random.normal(size=(1, nsample))
    
    # Combine diffusion and jump components
    X = A + J
    
    # Compute stock price trajectory
    S = S0 * np.exp(X)
    
    # Calculate option payoffs at maturity
    Vc_list[i] = np.exp(-r * T) * np.mean(np.maximum(S - K, 0))  # Call option
    Vp_list[i] = np.exp(-r * T) * np.mean(np.maximum(K - S, 0))  # Put option

# Calculate final call and put option prices
Vc = np.mean(Vc_list)
Vp = np.mean(Vp_list)

# Print results
print('The value of the call is: ' + str(round(Vc, 5)))
print('The value of the put is:  ' + str(round(Vp, 5)))


The value of the call is: 66.46502
The value of the put is:  151.28617


Based on a Merton jump diffusion model, the method use Monte Carlo simulation to forecast future prices of European call and put options. In order to estimate option prices, it first creates a number of random scenarios involving movements in stock prices, then adds diffusion and jump components, computes option payoffs for each scenario, and averages the results.